In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import db_connect
from tqdm import tqdm
from datetime import datetime

bu = 'chg'
table = 'stk'
date = '20250101'
chunk_size = 10000

chg_db3 = create_engine(db_connect.db_url_pstdb3)
q_chg_db3 = f"""
SELECT 
    cntnum, stmerch, cntdate, deptcode, deptname, subdeptcode, subdeptname,
    sku, sbc, ibc, bndname, prname, prmodel,
    soh, cntqnt, varianceqnt,
    extphycnt_retail, extphycnt_cost,
    extphy_retailvar, extphy_costvar,
    skutype, rpname 
FROM {bu}_{table}_this_year
"""
df_chg_db3 = pd.read_sql(q_chg_db3, chg_db3)

chg_db = create_engine(db_connect.db_url_pstdb)
q_chg_db = f"""
SELECT DISTINCT
    stmerch, cntdate, skutype, rpname
FROM {bu}_{table}
WHERE cntdate >= '{date}'
"""
df_chg_db = pd.read_sql(q_chg_db, chg_db)

# Anti-join
keys = ['stmerch', 'cntdate', 'skutype', 'rpname']
mask = ~df_chg_db3.set_index(keys).index.isin(df_chg_db.set_index(keys).index)
df = df_chg_db3[mask].reset_index(drop=True)

print(df.shape)
print(df.head(3))

# Insert with progress bar
with tqdm(total=len(df), desc="Inserting", unit="rows") as pbar:
    for start in range(0, len(df), chunk_size):
        chunk = df.iloc[start:start+chunk_size]
        chunk.to_sql(
            f"{bu}_{table}",
            chg_db,
            if_exists='append',
            index=False
        )
        pbar.update(len(chunk))

print(f"✅ Data {len(df)} inserted into {bu}_{table} at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


(2313235, 22)
            cntnum stmerch   cntdate deptcode              deptname  \
0  60922F080525002   60922  20250508       01  Construction & Tools   
1  60922F080525002   60922  20250508       01  Construction & Tools   
2  60922F080525002   60922  20250508       01  Construction & Tools   

  subdeptcode subdeptname       sku            sbc            ibc  ...  \
0        0106       Tools  60264728  8858833700428  2000602647281  ...   
1        0106       Tools  60256846  8858833905649  2000602568463  ...   
2        0106       Tools  60403220  8858833925067  2000604032207  ...   

  prmodel  soh cntqnt  varianceqnt  extphycnt_retail  extphycnt_cost  \
0  RH28-3  3.0    3.0          0.0           15870.0        11865.40   
1     958  3.0    3.0          0.0            3897.0         2913.65   
2   CD007  3.0    3.0          0.0            7770.0         5809.34   

   extphy_retailvar  extphy_costvar  skutype  rpname  
0               0.0             0.0  Consign    STK2  
1    

Inserting: 100%|██████████| 2313235/2313235 [08:52<00:00, 4344.69rows/s]

✅ Data 2313235 inserted into chg_stk at 2025-09-11 11:45:59
